In [ ]:
# IMportant check (softmatch)
    # def train_step(self, batch: dict, unlabeled_batch: dict) -> dict:
    #     data = batch['data']
    #     target = batch['target']
    
    #     data = data.to(self.device, non_blocking=True)
    #     if isinstance(tarsget, list):
    #         target = [i.to(self.device, non_blocking=True) for i in target]
    #     else:
    #         target = target.to(self.device, non_blocking=True)
    
    #     self.optimizer.zero_grad(set_to_none=True)
    
    #     with autocast(self.device.type, enabled=True) if self.device.type == 'cuda' else dummy_context():
    #         # ---- labeled部分（不改！）----
    #         output = self.network(data)
    #         loss_labeled = self.loss(output, target)  # ✅ 保持原本的逻辑
    
    #         # ---- unlabeled部分（只动这里）----
    #         if unlabeled_batch is not None:
    #             data_unlabeled = unlabeled_batch['data'].to(self.device, non_blocking=True)
    #             pseudo_logits = self.network(data_unlabeled)
    
    #             if isinstance(pseudo_logits, list):
    #                 pseudo_logits = pseudo_logits[0]  # ✅ 只在伪标签这边加！
    
    #             if self.label_manager.has_regions:
    #                 pseudo_target = (torch.sigmoid(pseudo_logits) > 0.5).float()
    #             else:
    #                 pseudo_target = pseudo_logits.argmax(1)
    
    #             pseudo_target = pseudo_target.detach()
    
    #             loss_unlabeled = nn.functional.cross_entropy(pseudo_logits, pseudo_target.long())
    
    #             total_loss = loss_labeled + 0.5 * loss_unlabeled
    #         else:
    #             total_loss = loss_labeled
    
    #     if self.grad_scaler is not None:
    #         self.grad_scaler.scale(total_loss).backward()
    #         self.grad_scaler.unscale_(self.optimizer)
    #         torch.nn.utils.clip_grad_norm_(self.network.parameters(), 12)
    #         self.grad_scaler.step(self.optimizer)
    #         self.grad_scaler.update()
    #     else:
    #         total_loss.backward()
    #         torch.nn.utils.clip_grad_norm_(self.network.parameters(), 12)
    #         self.optimizer.step()

    #     print(f"Epoch {self.current_epoch} | Loss Supervised: {loss_labeled.item():.4f} | Loss Unsupervised: {loss_unlabeled.item():.4f}" if unlabeled_batch is not None else f"Epoch {self.current_epoch} | Loss Supervised: {loss_labeled.item():.4f}")
    
    #     return {'loss': total_loss.detach().cpu().numpy()}

    def train_step(self, batch: dict, unlabeled_batch: dict) -> dict:
        data = batch['data']
        target = batch['target']
    
        data = data.to(self.device, non_blocking=True)
        if isinstance(target, list):
            target = [i.to(self.device, non_blocking=True) for i in target]
        else:
            target = target.to(self.device, non_blocking=True)
    
        self.optimizer.zero_grad(set_to_none=True)
    
        with autocast(self.device.type, enabled=True) if self.device.type == 'cuda' else dummy_context():
            # ---- Supervised part ----
            output = self.network(data)
            loss_labeled = self.loss(output, target)
    
            # ---- Unsupervised part ----
            if unlabeled_batch is not None:  # and self.current_epoch > 20:
                data_unlabeled = unlabeled_batch['data'].to(self.device, non_blocking=True)
    
                # Step 1: Weak augmentation
                pseudo_logits_w = self.network(data_unlabeled)
                if not isinstance(pseudo_logits_w, list):
                    pseudo_logits_w = [pseudo_logits_w]
    
                # Step 2: Strong augmentation
                data_unlabeled_strong, aug_info = self.strong_augmentor.augment(data_unlabeled)
                pseudo_logits_s = self.network(data_unlabeled_strong)
                if not isinstance(pseudo_logits_s, list):
                    pseudo_logits_s = [pseudo_logits_s]
    
                # Step 3: Compute pseudo labels for each resolution
                pseudo_targets_all = []
                masks_all = []
    
                for lw, ls in zip(pseudo_logits_w, pseudo_logits_s):
                    probs = torch.softmax(lw.detach(), dim=1)
                    max_probs, pseudo_target = probs.max(dim=1)  # [B, H, W]
                    mask = (max_probs > 0.5).float()
    
                    pseudo_target = pseudo_target.unsqueeze(1)
                    pseudo_target = self.strong_augmentor.deaugment(pseudo_target, aug_info).squeeze(1)
                    mask = self.strong_augmentor.deaugment(mask.unsqueeze(1), aug_info).squeeze(1)
    
                    pseudo_targets_all.append(pseudo_target)
                    masks_all.append(mask)
    
                # Step 4: Compute per-pixel CE loss with masking
                loss_unlabeled_all = []
                for ls, pt, m in zip(pseudo_logits_s, pseudo_targets_all, masks_all):
                    ce_map = F.cross_entropy(ls, pt, reduction='none')  # [B, H, W]
                    loss = (ce_map * m).sum() / (m.sum() + 1e-8)
                    loss_unlabeled_all.append(loss)
    
                loss_unlabeled = sum(loss_unlabeled_all) / len(loss_unlabeled_all)
                total_loss = loss_labeled + 1.0 * loss_unlabeled
    
            else:
                total_loss = loss_labeled
    
        if self.grad_scaler is not None:
            self.grad_scaler.scale(total_loss).backward()
            self.grad_scaler.unscale_(self.optimizer)
            torch.nn.utils.clip_grad_norm_(self.network.parameters(), 12)
            self.grad_scaler.step(self.optimizer)
            self.grad_scaler.update()
        else:
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.network.parameters(), 12)
            self.optimizer.step()
    
        return {'loss': total_loss.detach().cpu().numpy()}

In [ ]:
nnUNetv2_plan_and_preprocess -d 336 --verify_dataset_integrity


In [2]:
import os
import shutil

def copy_unique_files(folder1, folder2, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    files1 = set(os.listdir(folder1))
    files2 = set(os.listdir(folder2))

    unique_files = files2 - files1  # Files in folder2 not in folder1

    print(f"Found {len(unique_files)} unique files in folder2.")
    print(len(unique_files))

    for filename in unique_files:
        src_path = os.path.join(folder2, filename)
        dst_path = os.path.join(output_folder, filename)
        shutil.copy2(src_path, dst_path)
        print(f"Copied: {filename}")

# Example usage
folder1 = '/home/yi-yang/NNunet2/nnUNet_raw_data_base/nnUNet_raw/Dataset400_Livertumor100_AL/labelsTr'
folder2 = '/home/yi-yang/NNunet/nnUNet_raw_data_base/nnUNet_raw/Dataset327_Livertumor40/labelsTr'
output_folder = '/home/yi-yang/NNunet2/nnUNet_raw_data_base/nnUNet_raw/Dataset401_Livertumor1001_AL/labelsTr'

copy_unique_files(folder1, folder2, output_folder)


Found 100 unique files in folder2.
100
Copied: hepaticvessel_085.nii.gz
Copied: hepaticvessel_211.nii.gz
Copied: hepaticvessel_269.nii.gz
Copied: hepaticvessel_158.nii.gz
Copied: hepaticvessel_406.nii.gz
Copied: hepaticvessel_401.nii.gz
Copied: hepaticvessel_356.nii.gz
Copied: hepaticvessel_102.nii.gz
Copied: hepaticvessel_079.nii.gz
Copied: hepaticvessel_314.nii.gz
Copied: hepaticvessel_341.nii.gz
Copied: hepaticvessel_272.nii.gz
Copied: hepaticvessel_285.nii.gz
Copied: hepaticvessel_446.nii.gz
Copied: hepaticvessel_091.nii.gz
Copied: hepaticvessel_237.nii.gz
Copied: hepaticvessel_230.nii.gz
Copied: hepaticvessel_030.nii.gz
Copied: hepaticvessel_174.nii.gz
Copied: hepaticvessel_296.nii.gz
Copied: hepaticvessel_181.nii.gz
Copied: hepaticvessel_244.nii.gz
Copied: hepaticvessel_123.nii.gz
Copied: hepaticvessel_165.nii.gz
Copied: hepaticvessel_245.nii.gz
Copied: hepaticvessel_113.nii.gz
Copied: hepaticvessel_103.nii.gz
Copied: hepaticvessel_019.nii.gz
Copied: hepaticvessel_386.nii.gz
Copi

In [9]:
import os
import shutil
import nibabel as nib
import torch

def flip_and_save_nii_images(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if not filename.endswith('_0000.nii.gz'):
            continue

        input_path = os.path.join(input_folder, filename)
        base = filename.replace('_0000.nii.gz', '')

        # Copy original file directly
        out_orig = f"{base}_orig_0000.nii.gz"
        shutil.copy(input_path, os.path.join(output_folder, out_orig))

        # Load data once for flipping
        img = nib.load(input_path)
        data = img.get_fdata()
        affine = img.affine
        header = img.header

        image_tensor = torch.from_numpy(data)

        # Flip X
        flipped_x = torch.flip(image_tensor, dims=[-1]).numpy()
        out_x = f"{base}_flipx_0000.nii.gz"
        nib.save(nib.Nifti1Image(flipped_x, affine, header), os.path.join(output_folder, out_x))

        # Flip Y
        flipped_y = torch.flip(image_tensor, dims=[-2]).numpy()
        out_y = f"{base}_flipy_0000.nii.gz"
        nib.save(nib.Nifti1Image(flipped_y, affine, header), os.path.join(output_folder, out_y))

        print(f"Saved: {out_orig}, {out_x}, {out_y}")

# Example usage:
flip_and_save_nii_images('/home/yi-yang/NNunet/nnUNet_raw_data_base/NNunet_Probability/Ative_Learning_Max_Entropy/imagesTr_unlabel', '/home/yi-yang/NNunet/nnUNet_raw_data_base/NNunet_Probability/Active_Learning_Entropy+Dice/imagesTr_unlabel')


Saved: hepaticvessel_119_orig_0000.nii.gz, hepaticvessel_119_flipx_0000.nii.gz, hepaticvessel_119_flipy_0000.nii.gz
Saved: hepaticvessel_285_orig_0000.nii.gz, hepaticvessel_285_flipx_0000.nii.gz, hepaticvessel_285_flipy_0000.nii.gz
Saved: hepaticvessel_147_orig_0000.nii.gz, hepaticvessel_147_flipx_0000.nii.gz, hepaticvessel_147_flipy_0000.nii.gz
Saved: hepaticvessel_386_orig_0000.nii.gz, hepaticvessel_386_flipx_0000.nii.gz, hepaticvessel_386_flipy_0000.nii.gz
Saved: hepaticvessel_389_orig_0000.nii.gz, hepaticvessel_389_flipx_0000.nii.gz, hepaticvessel_389_flipy_0000.nii.gz


KeyboardInterrupt: 

In [1]:
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
import torch
import os

os.environ['nnUNet_raw'] = '/home/yi-yang/NNunet_MSD04/nnUNet_raw_data_base/nnUNet_raw'
os.environ['nnUNet_preprocessed'] = '/home/yi-yang/NNunet_MSD04/nnUNet_preprocessed'
os.environ['nnUNet_results'] = '/home/yi-yang/NNunet_MSD04/nnUNet_results'

predictor = nnUNetPredictor(device=torch.device('cuda'))

# Initialize with SoftMatch path
predictor.initialize_from_trained_model_folder(
    '/home/yi-yang/NNunet_KITS/nnUNet_results/Dataset347_KITS7-20250607T144202Z-1-001/Dataset347_KITS7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d',
    use_folds=(0,),
    checkpoint_name='checkpoint_best.pth'
)

print("Model class:", predictor.trainer_name)  # Must be nnUNetTrainer_SoftMatch
print("Model loaded successfully.")

input_folder = '/home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/output/test/images'  # Must contain _0000.nii.gz files
output_folder = '/home/yi-yang/NNunet_KITS/nnUNet_results/Dataset347_KITS7-20250607T144202Z-1-001/Dataset347_KITS7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d/SSL'
predictor.predict_from_files(
    list_of_lists_or_source_folder=input_folder,
    output_folder_or_list_of_truncated_output_files=output_folder,
    save_probabilities=False,
    overwrite=True,
    num_processes_preprocessing=2,
    num_processes_segmentation_export=2,
    folder_with_segs_from_prev_stage=None,
    num_parts=1,
    part_id=0
)

print("✅ Prediction finished. Check results in:", output_folder)


nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.
Model class: nnUNetTrainer_SoftMatch
Model loaded successfully.
There are 40 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 40 cases that I would like to predict

Predicting case_00013:
perform_everything_on

100%|███████████████████████████████████████████| 92/92 [00:03<00:00, 27.14it/s]


sending off prediction to background worker for resampling and export
done with case_00013

Predicting case_00023:
perform_everything_on_device: True


100%|█████████████████████████████████████████| 107/107 [00:02<00:00, 49.93it/s]


sending off prediction to background worker for resampling and export
done with case_00023

Predicting case_00028:
perform_everything_on_device: True


...

sending off prediction to background worker for resampling and export
done with case_00261

Predicting case_00262:
perform_everything_on_device: True


100%|███████████████████████████████████████████| 63/63 [00:01<00:00, 49.46it/s]


sending off prediction to background worker for resampling and export
done with case_00262

Predicting case_00288:
perform_everything_on_device: True


100%|███████████████████████████████████████| 1104/1104 [00:22<00:00, 48.32it/s]


sending off prediction to background worker for resampling and export
done with case_00288
✅ Prediction finished. Check results in: /home/yi-yang/NNunet_KITS/nnUNet_results/Dataset347_KITS7-20250607T144202Z-1-001/Dataset347_KITS7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d/SSL


In [2]:
import os

os.environ['nnUNet_raw'] = '/home/yi-yang/NNunet_MSD04/nnUNet_raw_data_base/nnUNet_raw'
os.environ['nnUNet_preprocessed'] = '/home/yi-yang/NNunet_MSD04/nnUNet_preprocessed'
os.environ['nnUNet_results'] = '/home/yi-yang/NNunet_MSD04/nnUNet_results'


from nnunetv2.evaluation.evaluate_predictions import compute_metrics_on_folder2

# Set your paths
folder_ref = '/home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/output/test/labelsTr'              # Ground truth labels
folder_pred = '/home/yi-yang/NNunet_KITS/nnUNet_results/Dataset347_KITS7-20250607T144202Z-1-001/Dataset347_KITS7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d/SSL'    # Predicted segmentations
dataset_json_file = '/home/yi-yang/NNunet_KITS/nnUNet_results/Dataset347_KITS7-20250607T144202Z-1-001/Dataset347_KITS7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d/dataset.json'   # From the nnUNet dataset folder
plans_file = '/home/yi-yang/NNunet_KITS/nnUNet_results/Dataset347_KITS7-20250607T144202Z-1-001/Dataset347_KITS7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d/plans.json'            # From the trained model folder
output_file = '/home/yi-yang/NNunet_KITS/nnUNet_results/Dataset347_KITS7-20250607T144202Z-1-001/Dataset347_KITS7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d/SSL/summary.json'  # Optional; will be created if not exists

# Run evaluation
compute_metrics_on_folder2(
    folder_ref,
    folder_pred,
    dataset_json_file,
    plans_file,
    output_file,
    num_processes=4,
    chill=True  # Set to False to raise error if files mismatch
)


Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer


In [1]:
import os
import json

# === Update these paths ===
train_folder = "/home/yi-yang/NNunet_LA/nnUNet_raw_data_base/nnUNet_raw/Dataset344_LA5/labelsTr"
val_folder = "/home/yi-yang/NNunet_LA/output/4/test2/labels"
output_file = "/home/yi-yang/NNunet_LA/nnUNet_raw_data_base/nnUNet_raw/Dataset344_LA5/splits_final.json"

# === Get case IDs from folder ===
def get_case_ids(folder):
    return sorted([
        f.replace(".nii.gz", "") for f in os.listdir(folder)
        if f.endswith(".nii.gz")
    ])

train_cases = get_case_ids(train_folder)
val_cases = get_case_ids(val_folder)

# === Create one fold ===
split = [
    {
        "train": train_cases,
        "val": val_cases
    }
]

# === Save JSON ===
with open(output_file, "w") as f:
    json.dump(split, f, indent=4)

print(f"✅ Done: 1-fold split with {len(train_cases)} train and {len(val_cases)} val cases.")


✅ Done: 1-fold split with 4 train and 20 val cases.


In [1]:
# import os
# import shutil

# # === Define your folders ===
# folder_1 = "/home/yi-yang/NNunet/nnUNet_results/Dataset335_Livertumor200-20250429T221324Z-1-002/nnUNetTrainer_250epochs__nnUNetPlans__2d/fold_0/validation"  # Source of filenames
# folder_2 = "/home/yi-yang/NNunet/nnUNet_raw_data_base/nnUNet_raw/Dataset325_Livertumor200/labelsTr"  # Source of files to copy from
# folder_3 = "/home/yi-yang/NNunet2/nnUNet_raw_data_base/Dataset328_Livertumor40/labelsTr_val"  # Destination to copy to

# # === Get filenames from folder_1 ===
# ref_filenames = {
#     f for f in os.listdir(folder_1) if f.endswith(".nii.gz")
# }

# # === Ensure destination exists ===
# os.makedirs(folder_3, exist_ok=True)

# # === Copy matching files from folder_2 to folder_3 ===
# copied = 0
# for fname in ref_filenames:
#     src = os.path.join(folder_2, fname)
#     dst = os.path.join(folder_3, fname)
#     if os.path.exists(src):
#         shutil.copy(src, dst)
#         copied += 1
#     else:
#         print(f"⚠️ File not found in folder_2: {fname}")

# print(f"✅ Copied {copied} files from folder_2 to folder_3.")


✅ Copied 40 files from folder_2 to folder_3.


In [15]:
# # === Set your paths ===
# folder_1 = "/home/yi-yang/NNunet2/nnUNet_raw_data_base/Dataset328_Livertumor40/labelsTr"  # Source of filenames
# folder_2 = "/home/yi-yang/NNunet2/nnUNet_raw_data_base/Dataset328_Livertumor40/imagesTr_left"  # Source of files to copy from
# folder_3 = "/home/yi-yang/NNunet2/nnUNet_raw_data_base/Dataset328_Livertumor40/imagesTr"  # Destination to copy to

# # === Get base names from folder_1 ===
# base_filenames = [
#     f.replace(".nii.gz", "") for f in os.listdir(folder_1) if f.endswith(".nii.gz")
# ]

# # === Make sure destination exists ===
# os.makedirs(folder_3, exist_ok=True)

# # === Copy matching _0000 files and rename ===
# copied = 0
# for base in base_filenames:
#     src = os.path.join(folder_2, f"{base}_0000.nii.gz")
#     dst = os.path.join(folder_3, f"{base}_0000.nii.gz")
#     if os.path.exists(src):
#         shutil.copy(src, dst)
#         copied += 1
#     else:
#         print(f"⚠️ Missing: {src}")

# print(f"✅ Copied and renamed {copied} files to: {folder_3}")

✅ Copied and renamed 40 files to: /home/yi-yang/NNunet2/nnUNet_raw_data_base/Dataset328_Livertumor40/imagesTr


In [14]:
# import os
# import shutil
# import random

# # === Set your folder paths ===
# folder_2 = "/home/yi-yang/NNunet2/nnUNet_raw_data_base/Dataset328_Livertumor40/labelsTr_left"  # destination (must exist and be empty)
# folder_1 = "/home/yi-yang/NNunet2/nnUNet_raw_data_base/Dataset328_Livertumor40/labelsTr"   # source of files

# # === Make sure destination exists ===
# os.makedirs(folder_1, exist_ok=True)

# # === List all .nii.gz files in folder_2 ===
# all_files = [f for f in os.listdir(folder_2) if f.endswith(".nii.gz")]

# # === Sanity check ===
# if len(all_files) < 40:
#     raise ValueError("Not enough files in folder_2 to pick 40 unique items.")

# # === Randomly select 40 unique files ===
# selected_files = random.sample(all_files, 40)

# # === Copy to folder_1 ===
# for f in selected_files:
#     shutil.copy(os.path.join(folder_2, f), os.path.join(folder_1, f))

# print(f"✅ Copied 40 random files to {folder_1}")


✅ Copied 40 random files to /home/yi-yang/NNunet2/nnUNet_raw_data_base/Dataset328_Livertumor40/labelsTr


In [27]:
import pickle
plans = pickle.load(open('/home/yi-yang/NNunet/nnUNet_preprocessed/Dataset329_Livertumor401/nnUNetPlans_2d/hepaticvessel_291.pkl', 'rb'))
print(plans)  # This tells you what it *tried* to load

{'sitk_stuff': {'spacing': (0.7851560115814209, 0.7851560115814209, 1.5), 'origin': (401.2147216796875, 401.2147216796875, 0.0), 'direction': (-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)}, 'spacing': [1.5, 0.7851560115814209, 0.7851560115814209], 'shape_before_cropping': (143, 512, 512), 'bbox_used_for_cropping': [[0, 143], [0, 512], [0, 512]], 'shape_after_cropping_and_before_resampling': (143, 512, 512), 'class_locations': {1: []}}


In [24]:
import os
import blosc2
import numpy as np

def compare_b2nd_folders(folder1, folder2):
    all_files = sorted([
        f for f in os.listdir(folder1)
        if f.endswith('.b2nd') and not f.endswith('_seg.b2nd')
    ])
    mismatch = []

    for fname in all_files:
        path1 = os.path.join(folder1, fname)
        path2 = os.path.join(folder2, fname)

        if not os.path.exists(path2):
            print(f"❌ {fname} missing in folder 2")
            mismatch.append(fname)
            continue

        # Load both files
        seg1 = blosc2.open(path1)[:]
        seg2 = blosc2.open(path2)[:]

        if not np.array_equal(seg1, seg2):
            print(f"⚠️ {fname} differs between folders")
            mismatch.append(fname)

    if not mismatch:
        print("✅ All .b2nd files are identical between the two folders.")
    else:
        print(f"\n🔍 {len(mismatch)} mismatched files found.")

# Example usage
folder1 = "/home/yi-yang/NNunet/nnUNet_preprocessed/Dataset329_Livertumor401/nnUNetPlans_2d"
folder2 = "/home/yi-yang/NNunet/nnUNet_preprocessed/Dataset329_Livertumor401/nnUNetPlans_2d111"
compare_b2nd_folders(folder1, folder2)


✅ All .b2nd files are identical between the two folders.


In [28]:
import os
import pickle

def check_class_locations_empty(pkl_folder):
    non_empty_files = []
    total_files = 0

    for fname in os.listdir(pkl_folder):
        if fname.endswith('.pkl'):
            total_files += 1
            full_path = os.path.join(pkl_folder, fname)
            with open(full_path, 'rb') as f:
                data = pickle.load(f)

            class_locations = data.get('class_locations', {})
            if any(len(v) > 0 for v in class_locations.values()):
                non_empty_files.append(fname)

    print(f"\nTotal .pkl files scanned: {total_files}")
    print(f"Files with non-empty class_locations: {len(non_empty_files)}")

    if non_empty_files:
        print("\nList of files with non-empty class_locations:")
        for f in non_empty_files:
            print(f)
    else:
        print("✅ All files have empty class_locations.")

# Example usage:
check_class_locations_empty('/home/yi-yang/NNunet/nnUNet_preprocessed/Dataset329_Livertumor401/nnUNetPlans_2d111')



Total .pkl files scanned: 160
Files with non-empty class_locations: 160

List of files with non-empty class_locations:
hepaticvessel_065.pkl
hepaticvessel_291.pkl
hepaticvessel_091.pkl
hepaticvessel_440.pkl
hepaticvessel_160.pkl
hepaticvessel_303.pkl
hepaticvessel_197.pkl
hepaticvessel_194.pkl
hepaticvessel_391.pkl
hepaticvessel_266.pkl
hepaticvessel_223.pkl
hepaticvessel_071.pkl
hepaticvessel_200.pkl
hepaticvessel_402.pkl
hepaticvessel_425.pkl
hepaticvessel_062.pkl
hepaticvessel_136.pkl
hepaticvessel_030.pkl
hepaticvessel_147.pkl
hepaticvessel_236.pkl
hepaticvessel_040.pkl
hepaticvessel_224.pkl
hepaticvessel_042.pkl
hepaticvessel_027.pkl
hepaticvessel_139.pkl
hepaticvessel_244.pkl
hepaticvessel_456.pkl
hepaticvessel_059.pkl
hepaticvessel_157.pkl
hepaticvessel_246.pkl
hepaticvessel_007.pkl
hepaticvessel_329.pkl
hepaticvessel_389.pkl
hepaticvessel_401.pkl
hepaticvessel_309.pkl
hepaticvessel_248.pkl
hepaticvessel_146.pkl
hepaticvessel_039.pkl
hepaticvessel_081.pkl
hepaticvessel_020.pkl


In [1]:
import h5py
import numpy as np

def check_slices(h5_path):
    with h5py.File(h5_path, 'r') as f:
        image = f['image'][:]
        label = f['label'][:]

        print(f"🖼 Image shape: {image.shape}")
        print(f"🏷  Label shape: {label.shape}")

        # Determine slice depth
        if image.ndim == 3:
            print(f"✅ Number of slices (depth): {image.shape[0]}")
        elif image.ndim == 2:
            print("⚠️ Image is 2D (single slice)")
        else:
            print("❌ Unexpected image shape")

        # Check label values
        unique_vals = np.unique(label)
        print(f"🔍 Unique label values: {unique_vals}")
        if np.array_equal(unique_vals, [0, 1]):
            print("✅ Label is binary (only 0 and 1).")
        else:
            print("⚠️ Label is not binary!")

# Example usage
check_slices('/home/yi-yang/Downloads/ACDC/ACDC/data/patient099_frame01.h5')


🖼 Image shape: (16, 224, 154)
🏷  Label shape: (16, 224, 154)
✅ Number of slices (depth): 16
🔍 Unique label values: [0 1 2 3]
⚠️ Label is not binary!


In [11]:
import nibabel as nib

def check_nii_shape(nii_path):
    img = nib.load(nii_path)
    data = img.get_fdata()
    print(f"🧠 File: {nii_path}")
    print(f"📐 Shape: {data.shape}")
    print(f"📏 Voxel spacing: {img.header.get_zooms()}")
    print(f"🔁 Affine:\n{img.affine}")

# Example usage
check_nii_shape('/home/yi-yang/Downloads/Resources/training/patient005/patient005_frame13_gt.nii.gz')


🧠 File: /home/yi-yang/Downloads/Resources/training/patient005/patient005_frame13_gt.nii.gz
📐 Shape: (256, 216, 10)
📏 Voxel spacing: (1.40625, 1.40625, 10.0)
🔁 Affine:
[[-1. -0.  0.  0.]
 [-0. -1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


In [3]:
import os
import shutil
import glob

import os
import shutil

def extract_all_files_from_patient_folders(list_file, base_dir, image_out, label_out):
    os.makedirs(image_out, exist_ok=True)
    os.makedirs(label_out, exist_ok=True)

    with open(list_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    patient_ids = sorted(set(line.split('_')[0] for line in lines))  # extract unique patient IDs

    for patient_id in patient_ids:
        patient_folder = os.path.join(base_dir, patient_id)
        if not os.path.isdir(patient_folder):
            print(f"❌ Folder not found: {patient_folder}")
            continue

        for fname in os.listdir(patient_folder):
            if fname.endswith('_4d.nii.gz'):
                continue  # skip 4d
            full_path = os.path.join(patient_folder, fname)

            if fname.endswith('_gt.nii.gz'):
                shutil.copy(full_path, os.path.join(label_out, fname))
                print(f"🏷️  Label copied: {fname}")
            elif fname.endswith('.nii.gz'):
                shutil.copy(full_path, os.path.join(image_out, fname))
                print(f"🖼  Image copied: {fname}")

# Example usage:
extract_all_files_from_patient_folders(
    list_file='/home/yi-yang/Downloads/ACDC/ACDC/test.list',
    base_dir='/home/yi-yang/Downloads/Resources/training',
    image_out='/home/yi-yang/ACDC/imagesTEST/',
    label_out='/home/yi-yang/ACDC/labelsTEST/'
)


🏷️  Label copied: patient001_frame12_gt.nii.gz
🏷️  Label copied: patient001_frame01_gt.nii.gz
🖼  Image copied: patient001_frame12.nii.gz
🖼  Image copied: patient001_frame01.nii.gz
🏷️  Label copied: patient007_frame01_gt.nii.gz
🖼  Image copied: patient007_frame01.nii.gz
🖼  Image copied: patient007_frame07.nii.gz
🏷️  Label copied: patient007_frame07_gt.nii.gz
🏷️  Label copied: patient008_frame13_gt.nii.gz
🖼  Image copied: patient008_frame01.nii.gz
🏷️  Label copied: patient008_frame01_gt.nii.gz
🖼  Image copied: patient008_frame13.nii.gz
🖼  Image copied: patient011_frame08.nii.gz
🏷️  Label copied: patient011_frame01_gt.nii.gz
🏷️  Label copied: patient011_frame08_gt.nii.gz
🖼  Image copied: patient011_frame01.nii.gz
🖼  Image copied: patient013_frame01.nii.gz
🏷️  Label copied: patient013_frame14_gt.nii.gz
🖼  Image copied: patient013_frame14.nii.gz
🏷️  Label copied: patient013_frame01_gt.nii.gz
🖼  Image copied: patient022_frame01.nii.gz
🖼  Image copied: patient022_frame11.nii.gz
🏷️  Label copi

In [4]:
import os
import shutil

def extract_unique_patient_ids(id_list_file):
    with open(id_list_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    # Extract only 'patientXXX' from each path like 'data/slices/patient099_frame01_slice_15.h5'
    return sorted(set(os.path.basename(line).split('_')[0] for line in lines))

def extract_by_patient_id_only(id_list_file, base_dir, image_out, label_out):
    os.makedirs(image_out, exist_ok=True)
    os.makedirs(label_out, exist_ok=True)

    patient_ids = extract_unique_patient_ids(id_list_file)

    for patient_id in patient_ids:
        folder = os.path.join(base_dir, patient_id)
        if not os.path.isdir(folder):
            print(f"❌ Folder not found: {folder}")
            continue

        for fname in os.listdir(folder):
            if not fname.endswith('.nii.gz') or '_4d' in fname:
                continue  # skip non-NIfTI and 4D files

            src = os.path.join(folder, fname)

            if fname.endswith('_gt.nii.gz'):
                shutil.copy(src, os.path.join(label_out, fname))
                print(f"🏷️  Copied label: {fname}")
            else:
                shutil.copy(src, os.path.join(image_out, fname))
                print(f"🖼  Copied image: {fname}")

# Example usage:

# Example usage:
extract_by_patient_id_only(
    id_list_file='/home/yi-yang/ACDC/unlabeled.txt',
    base_dir='/home/yi-yang/Downloads/Resources/training',
    image_out='/home/yi-yang/ACDC/unlabeled_images/',
    label_out='/home/yi-yang/ACDC/unlabeled_label/'
)


🖼  Copied image: patient003_frame15.nii.gz
🖼  Copied image: patient003_frame01.nii.gz
🏷️  Copied label: patient003_frame01_gt.nii.gz
🏷️  Copied label: patient003_frame15_gt.nii.gz
🖼  Copied image: patient004_frame01.nii.gz
🏷️  Copied label: patient004_frame15_gt.nii.gz
🏷️  Copied label: patient004_frame01_gt.nii.gz
🖼  Copied image: patient004_frame15.nii.gz
🖼  Copied image: patient005_frame13.nii.gz
🏷️  Copied label: patient005_frame13_gt.nii.gz
🖼  Copied image: patient005_frame01.nii.gz
🏷️  Copied label: patient005_frame01_gt.nii.gz
🖼  Copied image: patient006_frame01.nii.gz
🏷️  Copied label: patient006_frame16_gt.nii.gz
🏷️  Copied label: patient006_frame01_gt.nii.gz
🖼  Copied image: patient006_frame16.nii.gz
🖼  Copied image: patient009_frame01.nii.gz
🖼  Copied image: patient009_frame13.nii.gz
🏷️  Copied label: patient009_frame13_gt.nii.gz
🏷️  Copied label: patient009_frame01_gt.nii.gz
🖼  Copied image: patient010_frame13.nii.gz
🖼  Copied image: patient010_frame01.nii.gz
🏷️  Copied lab

In [17]:
import os

def remove_gt_suffix(folder):
    for fname in os.listdir(folder):
        if fname.endswith('_gt.nii.gz'):
            src = os.path.join(folder, fname)
            new_name = fname.replace('_gt.nii.gz', '.nii.gz')
            dst = os.path.join(folder, new_name)
            os.rename(src, dst)
            print(f"🔁 Renamed: {fname} → {new_name}")

# Example usage
remove_gt_suffix('/home/yi-yang/ACDC/labelsTEST')


🔁 Renamed: patient052_frame09_gt.nii.gz → patient052_frame09.nii.gz
🔁 Renamed: patient068_frame01_gt.nii.gz → patient068_frame01.nii.gz
🔁 Renamed: patient024_frame01_gt.nii.gz → patient024_frame01.nii.gz
🔁 Renamed: patient083_frame01_gt.nii.gz → patient083_frame01.nii.gz
🔁 Renamed: patient008_frame13_gt.nii.gz → patient008_frame13.nii.gz
🔁 Renamed: patient080_frame01_gt.nii.gz → patient080_frame01.nii.gz
🔁 Renamed: patient093_frame01_gt.nii.gz → patient093_frame01.nii.gz
🔁 Renamed: patient066_frame01_gt.nii.gz → patient066_frame01.nii.gz
🔁 Renamed: patient065_frame01_gt.nii.gz → patient065_frame01.nii.gz
🔁 Renamed: patient008_frame01_gt.nii.gz → patient008_frame01.nii.gz
🔁 Renamed: patient075_frame01_gt.nii.gz → patient075_frame01.nii.gz
🔁 Renamed: patient001_frame12_gt.nii.gz → patient001_frame12.nii.gz
🔁 Renamed: patient084_frame01_gt.nii.gz → patient084_frame01.nii.gz
🔁 Renamed: patient065_frame14_gt.nii.gz → patient065_frame14.nii.gz
🔁 Renamed: patient033_frame14_gt.nii.gz → patien

In [4]:
import os

def add_0000_suffix_to_images(folder):
    for fname in os.listdir(folder):
        if fname.endswith('.nii.gz') and '_0000' not in fname:
            src = os.path.join(folder, fname)
            base = fname[:-7]  # remove '.nii.gz'
            new_name = f"{base}_0000.nii.gz"
            dst = os.path.join(folder, new_name)
            os.rename(src, dst)
            print(f"📝 Renamed: {fname} → {new_name}")

# Example usage
add_0000_suffix_to_images('/home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset341_KITS02/imagesTr')


In [2]:
import os
import nibabel as nib
import numpy as np

def convert_gt_to_background_and_save(input_gt_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for fname in os.listdir(input_gt_folder):
        if fname.endswith('.nii.gz'):
            input_path = os.path.join(input_gt_folder, fname)
            output_path = os.path.join(output_folder, fname)

            img = nib.load(input_path)
            shape = img.get_fdata().shape
            affine = img.affine
            header = img.header

            # Create all-background mask (zeros)
            background_mask = np.zeros(shape, dtype=np.uint8)

            # Save to new location
            nib.save(nib.Nifti1Image(background_mask, affine, header), output_path)
            print(f"✅ Saved background-only: {fname}")

# Example usage
convert_gt_to_background_and_save(
    input_gt_folder='/home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset348_KITS9/labelsTr2',
    output_folder='/home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset348_KITS9/labelsTr'
)


✅ Saved background-only: case_00005.nii.gz
✅ Saved background-only: case_00295.nii.gz
✅ Saved background-only: case_00117.nii.gz
✅ Saved background-only: case_00044.nii.gz
✅ Saved background-only: case_00158.nii.gz
✅ Saved background-only: case_00130.nii.gz
✅ Saved background-only: case_00238.nii.gz
✅ Saved background-only: case_00183.nii.gz
✅ Saved background-only: case_00213.nii.gz
✅ Saved background-only: case_00147.nii.gz
✅ Saved background-only: case_00122.nii.gz
✅ Saved background-only: case_00294.nii.gz
✅ Saved background-only: case_00165.nii.gz
✅ Saved background-only: case_00076.nii.gz
✅ Saved background-only: case_00062.nii.gz
✅ Saved background-only: case_00188.nii.gz
✅ Saved background-only: case_00200.nii.gz
✅ Saved background-only: case_00131.nii.gz
✅ Saved background-only: case_00082.nii.gz
✅ Saved background-only: case_00138.nii.gz
✅ Saved background-only: case_00146.nii.gz
✅ Saved background-only: case_00187.nii.gz
✅ Saved background-only: case_00086.nii.gz
✅ Saved bac

In [1]:
import os
import shutil

# === Set your paths ===
folder1 = '/home/yi-yang/NNunet_MSD_Braintumor/nnUNet_raw_data_base/nnUNet_raw/Dataset384_BrainTumor4/labelsTr'  # Reference folder
folder2 = '/home/yi-yang/NNunet_MSD_Braintumor/nnUNet_raw_data_base/nnUNet_raw/Dataset381_BrainTumor2/labelsTr2'  # Source folder
output_folder = '/media/yi-yang/T7/MSD01/Task01_BrainTumour/output'  # Destination for unmatched files

os.makedirs(output_folder, exist_ok=True)

# Get filenames (not full paths)
folder1_files = set(os.listdir(folder1))
folder2_files = set(os.listdir(folder2))

# Find files in folder2 that are NOT in folder1
unique_files = folder2_files - folder1_files

# Copy unmatched files to output folder
for filename in unique_files:
    src = os.path.join(folder2, filename)
    dst = os.path.join(output_folder, filename)
    shutil.copy2(src, dst)
    print(f"Copied: {filename}")


Copied: BRATS_361.nii.gz
Copied: BRATS_045.nii.gz
Copied: BRATS_140.nii.gz
Copied: BRATS_203.nii.gz
Copied: BRATS_222.nii.gz
Copied: BRATS_248.nii.gz
Copied: BRATS_452.nii.gz
Copied: BRATS_253.nii.gz
Copied: BRATS_389.nii.gz
Copied: BRATS_163.nii.gz
Copied: BRATS_125.nii.gz
Copied: BRATS_042.nii.gz
Copied: BRATS_432.nii.gz
Copied: BRATS_474.nii.gz
Copied: BRATS_419.nii.gz
Copied: BRATS_244.nii.gz
Copied: BRATS_442.nii.gz
Copied: BRATS_268.nii.gz
Copied: BRATS_313.nii.gz
Copied: BRATS_043.nii.gz
Copied: BRATS_284.nii.gz


In [2]:
import os
import shutil

# === Update these paths ===
folder1 = "/home/yi-yang/NNunet_SOTA3/nnUNet_raw_data_base/nnUNet_raw/label"
folder2 = "/home/yi-yang/NNunet_SOTA3/nnUNet_raw_data_base/nnUNet_raw/Dataset328_ACDC7/labelsTr"
output_folder = "/home/yi-yang/NNunet_SOTA3/nnUNet_raw_data_base/nnUNet_raw/label_U"

os.makedirs(output_folder, exist_ok=True)

# Get file names in Folder 1
folder1_files = set(os.listdir(folder1))

# Loop through Folder 2
for file in os.listdir(folder2):
    if file not in folder1_files:
        src = os.path.join(folder2, file)
        dst = os.path.join(output_folder, file)
        shutil.copy2(src, dst)
        print(f"Copied: {file}")


Copied: patient002_frame01.nii.gz
Copied: patient045_frame01.nii.gz
Copied: patient002_frame12.nii.gz
Copied: patient085_frame01.nii.gz
Copied: patient082_frame01.nii.gz
Copied: patient030_frame12.nii.gz
Copied: patient045_frame13.nii.gz
Copied: patient078_frame01.nii.gz
Copied: patient039_frame10.nii.gz
Copied: patient021_frame13.nii.gz
Copied: patient019_frame11.nii.gz
Copied: patient049_frame01.nii.gz
Copied: patient085_frame09.nii.gz
Copied: patient082_frame07.nii.gz
Copied: patient087_frame10.nii.gz
Copied: patient021_frame01.nii.gz
Copied: patient030_frame01.nii.gz
Copied: patient019_frame01.nii.gz
Copied: patient058_frame14.nii.gz
Copied: patient028_frame09.nii.gz
Copied: patient058_frame01.nii.gz
Copied: patient100_frame13.nii.gz
Copied: patient049_frame11.nii.gz
Copied: patient028_frame01.nii.gz
Copied: patient039_frame01.nii.gz
Copied: patient087_frame01.nii.gz
Copied: patient078_frame09.nii.gz
Copied: patient100_frame01.nii.gz


In [5]:
import os
import nibabel as nib
import numpy as np

def convert_labels_to_binary(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for fname in os.listdir(input_folder):
        if fname.endswith('.nii.gz') and not fname.startswith('._'):
            input_path = os.path.join(input_folder, fname)
            output_path = os.path.join(output_folder, fname)

            # Load NIfTI
            nii = nib.load(input_path)
            data = nii.get_fdata()

            # Convert all non-zero to 1
            binary_data = (data > 0).astype(np.uint8)

            # Save binary label
            binary_nii = nib.Nifti1Image(binary_data, nii.affine, nii.header)
            nib.save(binary_nii, output_path)
            print(f"✅ Converted and saved: {fname}")

convert_labels_to_binary('/home/yi-yang/NNunet_MSD_Braintumor/nnUNet_raw_data_base/nnUNet_raw/output/test/labels', '/home/yi-yang/NNunet_MSD_Braintumor/nnUNet_raw_data_base/nnUNet_raw/output/test/labels2')

✅ Converted and saved: BRATS_255.nii.gz
✅ Converted and saved: BRATS_416.nii.gz
✅ Converted and saved: BRATS_234.nii.gz
✅ Converted and saved: BRATS_092.nii.gz
✅ Converted and saved: BRATS_118.nii.gz
✅ Converted and saved: BRATS_379.nii.gz
✅ Converted and saved: BRATS_215.nii.gz
✅ Converted and saved: BRATS_140.nii.gz
✅ Converted and saved: BRATS_309.nii.gz
✅ Converted and saved: BRATS_442.nii.gz
✅ Converted and saved: BRATS_047.nii.gz
✅ Converted and saved: BRATS_315.nii.gz
✅ Converted and saved: BRATS_271.nii.gz
✅ Converted and saved: BRATS_021.nii.gz
✅ Converted and saved: BRATS_424.nii.gz
✅ Converted and saved: BRATS_275.nii.gz
✅ Converted and saved: BRATS_382.nii.gz
✅ Converted and saved: BRATS_013.nii.gz
✅ Converted and saved: BRATS_259.nii.gz
✅ Converted and saved: BRATS_409.nii.gz
✅ Converted and saved: BRATS_356.nii.gz
✅ Converted and saved: BRATS_451.nii.gz
✅ Converted and saved: BRATS_203.nii.gz
✅ Converted and saved: BRATS_474.nii.gz
✅ Converted and saved: BRATS_125.nii.gz


In [1]:
import os
import nibabel as nib
import numpy as np

input_folder = "/home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr2"
output_folder = "/home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith(".nii.gz"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        img = nib.load(input_path)
        data = img.get_fdata()
        affine = img.affine
        header = img.header

        # Convert: only index 2 becomes 1, all others become 0
        new_data = np.where(data == 2, 1, 0).astype(np.uint8)

        nib.save(nib.Nifti1Image(new_data, affine, header), output_path)
        print(f"✅ Saved: {output_path}")


✅ Saved: /home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr/case_00087.nii.gz
✅ Saved: /home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr/case_00038.nii.gz
✅ Saved: /home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr/case_00166.nii.gz
✅ Saved: /home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr/case_00045.nii.gz
✅ Saved: /home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr/case_00101.nii.gz
✅ Saved: /home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr/case_00112.nii.gz
✅ Saved: /home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr/case_00172.nii.gz
✅ Saved: /home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr/case_00018.nii.gz
✅ Saved: /home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/Dataset347_KITS7/labelsTr/case_00290.nii.gz
✅

In [1]:
! pip install seg-metrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 24.3 MB/s eta 0:00:00 MB/s eta 0:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.2 MB/s eta 0:00:00


In [9]:
# import seg_metrics.seg_metrics as sg

# labels = [1, 2, 3]  # Replace with your actual class labels
# gdth_path = '/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST'
# pred_path = '/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/NNunet_SOTA/nnUNet_results/Dataset328_ACDC7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d_90.06/sss'
# csv_file = '/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/metrics2.csv'

# metrics = sg.write_metrics(labels=labels,
#                            gdth_path=gdth_path,
#                            pred_path=pred_path,
#                            csv_file=csv_file)
# print(metrics)

NiftiImageIO (0x22b7dc40): /media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST/patient001_frame01.nii.gz has unexpected scales in sform

NiftiImageIO (0x22b7dc40): /media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST/patient001_frame01.nii.gz has unexpected scales in sform

NiftiImageIO (0x22b7dc40): /media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST/patient001_frame12.nii.gz has unexpected scales in sform

NiftiImageIO (0x22b7dc40): /media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST/patient001_frame12.nii.gz has unexpected scales in sform

NiftiImageIO (0x22b7dc40): /media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST/patient007_frame01.nii.gz has unexpected scales in sform

NiftiImageIO (0x22b7dc40): /media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST/patient007_frame01.nii.gz has unexpected scales in sform

NiftiImageIO (0x22b7dc40): /media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST/p

[{'label': [1, 2, 3], 'dice': [0.9338989042064334, 0.8369162823926739, 0.9474636191729015], 'jaccard': [0.8759946949602122, 0.7195666741121287, 0.9001718481487267], 'precision': [0.9447523690327194, 0.7432229784288845, 0.9428898707249223], 'recall': [0.9232919797309104, 0.957639714625446, 0.9520819563780568], 'fpr': [0.0005646548022154935, 0.004075191493724278, 0.001290546836866005], 'fnr': [0.07670802026908964, 0.0423602853745541, 0.04791804362194316], 'vs': [-0.022976316719688937, 0.25212703773462364, 0.00970155389295404], 'hd': [13.975424766540527, 16.083887100219727, 17.1875], 'msd': [0.4176099756288867, 0.8410382406764815, 0.7916886556372362], 'mdsd': [0.0, 0.0, 0.0], 'stdsd': [1.6037788759733242, 2.418405547806012, 2.201828295801658], 'hd95': [1.5625, 3.493856191635132, 4.6875], 'filename': '/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/NNunet_SOTA/nnUNet_results/Dataset328_ACDC7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d_90.06/sss/patient001_frame01.nii.gz'}, {'label': [1, 2, 

In [10]:
! pip install MedPy

  Preparing metadata (setup.py) ... done
  Created wheel for MedPy: filename=MedPy-0.5.2-py3-none-any.whl size=224775 sha256=f12ad93e468c00909da611d04a38900f161ec8800f45e2a62bda6032981686a3
  Stored in directory: /home/yi-yang/.cache/pip/wheels/d4/33/ed/aaac5a347fb8d41679ca515b8f5c49dfdf49be15bdbb9a905d
Successfully built MedPy


In [7]:
import os
import nibabel as nib

def get_spacing_dict(folder):
    spacing_dict = {}
    for fname in os.listdir(folder):
        if fname.endswith(".nii.gz"):
            path = os.path.join(folder, fname)
            try:
                spacing = nib.load(path).header.get_zooms()
                spacing_dict[fname] = spacing
            except Exception as e:
                print(f"Error reading {fname}: {e}")
    return spacing_dict

def compare_spacings(gt_folder, pred_folder):
    gt_spacings = get_spacing_dict(gt_folder)
    pred_spacings = get_spacing_dict(pred_folder)

    for fname in sorted(gt_spacings):
        if fname in pred_spacings:
            gt_spacing = gt_spacings[fname]
            pred_spacing = pred_spacings[fname]
            if gt_spacing != pred_spacing:
                print(f"[DIFF] {fname}: GT {gt_spacing} vs PRED {pred_spacing}")
            else:
                print(f"[MATCH] {fname}: {gt_spacing}")
        else:
            print(f"[MISSING IN PRED] {fname}")

# Example usage:
gt_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST"
pred_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/NNunet_SOTA/nnUNet_results/Dataset328_ACDC7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d_90.06/sss"
compare_spacings(gt_folder, pred_folder)


[MATCH] patient001_frame01.nii.gz: (1.5625, 1.5625, 10.0)
[MATCH] patient001_frame12.nii.gz: (1.5625, 1.5625, 10.0)
[MATCH] patient007_frame01.nii.gz: (1.875, 1.875, 10.0)
[MATCH] patient007_frame07.nii.gz: (1.875, 1.875, 10.0)
[MATCH] patient008_frame01.nii.gz: (1.5625, 1.5625, 10.0)
[MATCH] patient008_frame13.nii.gz: (1.5625, 1.5625, 10.0)
[MATCH] patient011_frame01.nii.gz: (1.48438, 1.48438, 10.0)
[MATCH] patient011_frame08.nii.gz: (1.48438, 1.48438, 10.0)
[MATCH] patient013_frame01.nii.gz: (1.44531, 1.44531, 10.0)
[MATCH] patient013_frame14.nii.gz: (1.44531, 1.44531, 10.0)
[MATCH] patient022_frame01.nii.gz: (1.40625, 1.40625, 10.0)
[MATCH] patient022_frame11.nii.gz: (1.40625, 1.40625, 10.0)
[MATCH] patient024_frame01.nii.gz: (1.5625, 1.5625, 10.0)
[MATCH] patient024_frame09.nii.gz: (1.5625, 1.5625, 10.0)
[MATCH] patient033_frame01.nii.gz: (1.5625, 1.5625, 10.0)
[MATCH] patient033_frame14.nii.gz: (1.5625, 1.5625, 10.0)
[MATCH] patient052_frame01.nii.gz: (1.48438, 1.48438, 10.0)
[MAT

In [1]:
import os
import nibabel as nib
import numpy as np
from medpy import metric

# Set your folders
gt_folder = "/home/yi-yang/NNunet_LA/output/4/test2/labels"
pred_folder = "/home/yi-yang/NNunet_LA/nnUNet_results/Dataset340_LA1/Dataset340_LA1-20250612T133621Z-1-001/Dataset340_LA1_5e-3+3e-4/nnUNetTrainer_SoftMatch__nnUNetPlans__3d_fullres/fold_0/validation"
# gt_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST"
# pred_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/NNunet_SOTA3/nnUNet_results/Dataset330_ACDC3/nnUNetTrainer_SoftMatch__nnUNetPlans__2d_89.56/SSL"



# Define your label classes
label_classes = [1]

# Initialize results storage
results = {label: [] for label in label_classes}

# Loop through all matching files
for fname in sorted(os.listdir(gt_folder)):
    if not fname.endswith('.nii.gz'):
        continue

    gt_path = os.path.join(gt_folder, fname)
    pred_path = os.path.join(pred_folder, fname)

    if not os.path.exists(pred_path):
        print(f"[SKIP] Missing prediction: {fname}")
        continue

    # Load NIfTI files
    gt_img = nib.load(gt_path).get_fdata()
    pred_img = nib.load(pred_path).get_fdata()

    # Ensure same shape
    if gt_img.shape != pred_img.shape:
        print(f"[ERROR] Shape mismatch: {fname}")
        continue

    # Per class evaluation
    for label in label_classes:
        gt_binary = (gt_img == label).astype(np.uint8)
        pred_binary = (pred_img == label).astype(np.uint8)

        if np.sum(gt_binary) == 0 and np.sum(pred_binary) == 0:
            continue  # No instance in either
        elif np.sum(gt_binary) == 0 or np.sum(pred_binary) == 0:
            hd95 = np.nan  # Undefined
            asd = np.nan
        else:
            try:
                hd95 = metric.binary.hd95(pred_binary, gt_binary)
                asd = metric.binary.asd(pred_binary, gt_binary)
            except:
                hd95 = np.nan
                asd = np.nan

        results[label].append({'file': fname, 'hd95': hd95, 'asd': asd})

# Report mean per class
for label in label_classes:
    hd95_vals = [x['hd95'] for x in results[label] if not np.isnan(x['hd95'])]
    asd_vals = [x['asd'] for x in results[label] if not np.isnan(x['asd'])]

    if hd95_vals:
        print(f"Label {label}: Mean 95HD = {np.mean(hd95_vals):.2f} voxels, Mean ASD = {np.mean(asd_vals):.2f} voxels")
    else:
        print(f"Label {label}: No valid entries")


Label 1: Mean 95HD = 5.58 voxels, Mean ASD = 1.83 voxels


In [2]:
import os
import nibabel as nib
import numpy as np
from medpy import metric

# Set your folders
gt_folder = "/home/yi-yang/NNunet_LA/output/4/test2/labels"
pred_folder = "/home/yi-yang/NNunet_LA/nnUNet_results/Dataset342_LA3/nnUNetTrainer_SoftMatch__nnUNetPlans__3d_fullres_3e-3+3e-4/fold_0/validation"
# gt_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST"
# pred_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/NNunet_SOTA3/nnUNet_results/Dataset330_ACDC3/nnUNetTrainer_SoftMatch__nnUNetPlans__2d_89.56/SSL"



# Define your label classes
label_classes = [1]

# Initialize results storage
results = {label: [] for label in label_classes}

# Loop through all matching files
for fname in sorted(os.listdir(gt_folder)):
    if not fname.endswith('.nii.gz'):
        continue

    gt_path = os.path.join(gt_folder, fname)
    pred_path = os.path.join(pred_folder, fname)

    if not os.path.exists(pred_path):
        print(f"[SKIP] Missing prediction: {fname}")
        continue

    # Load NIfTI files
    gt_img = nib.load(gt_path).get_fdata()
    pred_img = nib.load(pred_path).get_fdata()

    # Ensure same shape
    if gt_img.shape != pred_img.shape:
        print(f"[ERROR] Shape mismatch: {fname}")
        continue

    # Per class evaluation
    for label in label_classes:
        gt_binary = (gt_img == label).astype(np.uint8)
        pred_binary = (pred_img == label).astype(np.uint8)

        if np.sum(gt_binary) == 0 and np.sum(pred_binary) == 0:
            continue  # No instance in either
        elif np.sum(gt_binary) == 0 or np.sum(pred_binary) == 0:
            hd95 = np.nan  # Undefined
            asd = np.nan
        else:
            try:
                hd95 = metric.binary.hd95(pred_binary, gt_binary)
                asd = metric.binary.asd(pred_binary, gt_binary)
            except:
                hd95 = np.nan
                asd = np.nan

        results[label].append({'file': fname, 'hd95': hd95, 'asd': asd})

# Report mean per class
for label in label_classes:
    hd95_vals = [x['hd95'] for x in results[label] if not np.isnan(x['hd95'])]
    asd_vals = [x['asd'] for x in results[label] if not np.isnan(x['asd'])]

    if hd95_vals:
        print(f"Label {label}: Mean 95HD = {np.mean(hd95_vals):.2f} voxels, Mean ASD = {np.mean(asd_vals):.2f} voxels")
    else:
        print(f"Label {label}: No valid entries")


Label 1: Mean 95HD = 5.78 voxels, Mean ASD = 1.85 voxels


In [4]:
import os
import nibabel as nib
import numpy as np
from medpy import metric

# Set your folders
gt_folder = "/home/yi-yang/NNunet_KITS/nnUNet_raw_data_base/nnUNet_raw/output/test/labelsTr"
pred_folder = "/home/yi-yang/NNunet_KITS/nnUNet_results/Dataset342_KITS03/nnUNetTrainer_SoftMatch__nnUNetPlans__2d_LR_0.01/SSL"
# gt_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST"
# pred_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/NNunet_SOTA3/nnUNet_results/Dataset330_ACDC3/nnUNetTrainer_SoftMatch__nnUNetPlans__2d_89.56/SSL"



# Define your label classes
label_classes = [1]

# Initialize results storage
results = {label: [] for label in label_classes}

# Loop through all matching files
for fname in sorted(os.listdir(gt_folder)):
    if not fname.endswith('.nii.gz'):
        continue

    gt_path = os.path.join(gt_folder, fname)
    pred_path = os.path.join(pred_folder, fname)

    if not os.path.exists(pred_path):
        print(f"[SKIP] Missing prediction: {fname}")
        continue

    # Load NIfTI files
    gt_img = nib.load(gt_path).get_fdata()
    pred_img = nib.load(pred_path).get_fdata()

    # Ensure same shape
    if gt_img.shape != pred_img.shape:
        print(f"[ERROR] Shape mismatch: {fname}")
        continue

    # Per class evaluation
    for label in label_classes:
        gt_binary = (gt_img == label).astype(np.uint8)
        pred_binary = (pred_img == label).astype(np.uint8)

        if np.sum(gt_binary) == 0 and np.sum(pred_binary) == 0:
            continue  # No instance in either
        elif np.sum(gt_binary) == 0 or np.sum(pred_binary) == 0:
            hd95 = np.nan  # Undefined
            asd = np.nan
        else:
            try:
                hd95 = metric.binary.hd95(pred_binary, gt_binary)
                asd = metric.binary.asd(pred_binary, gt_binary)
            except:
                hd95 = np.nan
                asd = np.nan

        results[label].append({'file': fname, 'hd95': hd95, 'asd': asd})

# Report mean per class
for label in label_classes:
    hd95_vals = [x['hd95'] for x in results[label] if not np.isnan(x['hd95'])]
    asd_vals = [x['asd'] for x in results[label] if not np.isnan(x['asd'])]

    if hd95_vals:
        print(f"Label {label}: Mean 95HD = {np.mean(hd95_vals):.2f} voxels, Mean ASD = {np.mean(asd_vals):.2f} voxels")
    else:
        print(f"Label {label}: No valid entries")


Label 1: Mean 95HD = 50.26 voxels, Mean ASD = 22.76 voxels


In [12]:
import os
import nibabel as nib
import numpy as np
from medpy import metric

# Set your folders
gt_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/ACDC/labelsTEST"
pred_folder = "/media/yi-yang/T7/Result/NNunet_SSL_AL/ACDC_SOTA7/NNunet_SOTA/nnUNet_results/Dataset328_ACDC7/nnUNetTrainer_SoftMatch__nnUNetPlans__2d_90.06/sss"

# Define your label classes
label_classes = [1, 2, 3]

# Initialize results storage
results = {label: [] for label in label_classes}

# Loop through all matching files
for fname in sorted(os.listdir(gt_folder)):
    if not fname.endswith('.nii.gz'):
        continue

    gt_path = os.path.join(gt_folder, fname)
    pred_path = os.path.join(pred_folder, fname)

    if not os.path.exists(pred_path):
        print(f"[SKIP] Missing prediction: {fname}")
        continue

    # Load NIfTI files
    gt_nib = nib.load(gt_path)
    pred_nib = nib.load(pred_path)

    gt_img = gt_nib.get_fdata()
    pred_img = pred_nib.get_fdata()

    # Get voxel spacing (in mm)
    spacing = gt_nib.header.get_zooms()  # (x, y, z)

    # Ensure same shape
    if gt_img.shape != pred_img.shape:
        print(f"[ERROR] Shape mismatch: {fname}")
        continue

    for label in label_classes:
        gt_binary = (gt_img == label).astype(np.uint8)
        pred_binary = (pred_img == label).astype(np.uint8)

        if np.sum(gt_binary) == 0 and np.sum(pred_binary) == 0:
            continue
        elif np.sum(gt_binary) == 0 or np.sum(pred_binary) == 0:
            hd95 = np.nan
            asd = np.nan
        else:
            try:
                hd95 = metric.binary.hd95(pred_binary, gt_binary, voxelspacing=spacing)
                asd = metric.binary.asd(pred_binary, gt_binary, voxelspacing=spacing)
            except:
                hd95 = np.nan
                asd = np.nan

        results[label].append({'file': fname, 'hd95': hd95, 'asd': asd})

# Report mean per class
for label in label_classes:
    hd95_vals = [x['hd95'] for x in results[label] if not np.isnan(x['hd95'])]
    asd_vals = [x['asd'] for x in results[label] if not np.isnan(x['asd'])]

    if hd95_vals:
        print(f"Label {label}: Mean 95HD = {np.mean(hd95_vals):.2f} mm, Mean ASD = {np.mean(asd_vals):.2f} mm")
    else:
        print(f"Label {label}: No valid entries")


Label 1: Mean 95HD = 4.93 mm, Mean ASD = 1.36 mm
Label 2: Mean 95HD = 3.68 mm, Mean ASD = 1.02 mm
Label 3: Mean 95HD = 4.66 mm, Mean ASD = 1.04 mm


In [4]:
import os
import shutil

# Define your folders
folder1 = "/home/yi-yang/NNunet_MSDLT/nnUNet_raw_data_base/nnUNet_raw/Dataset345_MSD_086/labelsTr2"  # source folder
folder2 = "/home/yi-yang/NNunet_MSDLT/nnUNet_raw_data_base/nnUNet_raw/Dataset346_MSD_087/labels"  # filenames to match
folder3 = "/home/yi-yang/NNunet_MSDLT/nnUNet_raw_data_base/nnUNet_raw/Dataset346_MSD_087/labels2"  # destination folder

# Make sure destination folder exists
os.makedirs(folder3, exist_ok=True)

# Get list of target filenames (without full path)
target_filenames = set(os.listdir(folder2))

# Go through Folder 1 and copy matching files
for file_name in os.listdir(folder1):
    if file_name in target_filenames:
        src = os.path.join(folder1, file_name)
        dst = os.path.join(folder3, file_name)
        shutil.copy2(src, dst)

print("✅ Done copying matching files.")


✅ Done copying matching files.


In [5]:
import os
import shutil

def extract_files_by_suffix(txt_path, source_folder, destination_folder):
    # Read last-4-digit suffixes from txt
    with open(txt_path, 'r') as f:
        suffixes = [line.strip().replace('.jpg', '')[-4:] for line in f if line.strip()]

    # Create output folder
    os.makedirs(destination_folder, exist_ok=True)

    # Search and copy matching files
    for fname in os.listdir(source_folder):
        if fname.lower().endswith('.jpg') and fname[-8:-4] in suffixes:
            full_src = os.path.join(source_folder, fname)
            full_dst = os.path.join(destination_folder, fname)
            shutil.copy(full_src, full_dst)

# Example usage:
txt_file = '/home/yi-yang/Downloads/test_100.txt'  # Your .txt file with names like '1196.jpg'
src_folder = '/home/yi-yang/Downloads/ISIC2018_Task1-2_Training_Input'
dst_folder = '/home/yi-yang/NNunet_isic/train'

extract_files_by_suffix(txt_file, src_folder, dst_folder)


In [14]:
def find_overlapping_filenames(txt_file1, txt_file2):
    def load_suffixes(txt_path):
        with open(txt_path, 'r') as f:
            return set(line.strip().replace('.jpg', '')[-4:] for line in f if line.strip())

    # Load last-4-digit suffixes
    suffixes1 = load_suffixes(txt_file1)
    suffixes2 = load_suffixes(txt_file2)

    # Find overlap
    overlap = suffixes1.intersection(suffixes2)

    print(f"Number of overlapping files: {len(overlap)}")
    if overlap:
        print("Overlapping file suffixes (last 4 digits):")
        for name in sorted(overlap):
            print(name)

# Example usage
txt1 = '/home/yi-yang/Downloads/test_100.txt'
txt2 = '/home/yi-yang/Downloads/train_259.txt'

find_overlapping_filenames(txt1, txt2)


Number of overlapping files: 16
Overlapping file suffixes (last 4 digits):
0001
0008
0013
0017
0020
0026
0041
0043
0048
0055
0071
0072
0075
0078
0081
0086


In [1]:
from PIL import Image
import os

def convert_jpg_to_rgb_png(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for fname in os.listdir(input_folder):
        if fname.lower().endswith('.jpg'):
            input_path = os.path.join(input_folder, fname)
            output_name = os.path.splitext(fname)[0] + '.png'
            output_path = os.path.join(output_folder, output_name)

            try:
                img = Image.open(input_path).convert('RGB')  # force RGB mode
                img.save(output_path)
                print(f"Converted: {fname} → {output_name}")
            except Exception as e:
                print(f"Failed to convert {fname}: {e}")

# Example usage
convert_jpg_to_rgb_png(
    input_folder='/home/yi-yang/NNunet_isic/output/ISIC2018_Task1-2_Validation_Input',
    output_folder='/home/yi-yang/NNunet_isic/output/ISIC2018_Task1-2_Validation_Input_png'
)


Converted: ISIC_0012903.jpg → ISIC_0012903.png
Converted: ISIC_0014291.jpg → ISIC_0014291.png
Converted: ISIC_0011386.jpg → ISIC_0011386.png
Converted: ISIC_0000081.jpg → ISIC_0000081.png
Converted: ISIC_0011163.jpg → ISIC_0011163.png
Converted: ISIC_0003005.jpg → ISIC_0003005.png
Converted: ISIC_0008998.jpg → ISIC_0008998.png
Converted: ISIC_0012151.jpg → ISIC_0012151.png
Converted: ISIC_0012235.jpg → ISIC_0012235.png
Converted: ISIC_0000013.jpg → ISIC_0000013.png
Converted: ISIC_0015948.jpg → ISIC_0015948.png
Converted: ISIC_0015158.jpg → ISIC_0015158.png
Converted: ISIC_0008029.jpg → ISIC_0008029.png
Converted: ISIC_0012856.jpg → ISIC_0012856.png
Converted: ISIC_0000202.jpg → ISIC_0000202.png
Converted: ISIC_0000058.jpg → ISIC_0000058.png
Converted: ISIC_0000150.jpg → ISIC_0000150.png
Converted: ISIC_0015944.jpg → ISIC_0015944.png
Converted: ISIC_0006800.jpg → ISIC_0006800.png
Converted: ISIC_0014169.jpg → ISIC_0014169.png
Converted: ISIC_0000499.jpg → ISIC_0000499.png
Converted: IS

In [5]:
from PIL import Image
import numpy as np
import os

def convert_labels_255_to_1(folder):
    for fname in os.listdir(folder):
        if fname.endswith('.png'):
            path = os.path.join(folder, fname)
            try:
                label = np.array(Image.open(path))

                # Convert 255 → 1
                fixed_label = (label > 0).astype(np.uint8)

                # Save back as .png
                Image.fromarray(fixed_label).save(path)
                print(f"✅ Fixed: {fname}")
            except Exception as e:
                print(f"❌ Failed: {fname} ({e})")

# Example usage
convert_labels_255_to_1('/home/yi-yang/NNunet_isic/nnUNet_raw_data_base/nnUNet_raw/Dataset330_ISIC1/labelsTr')


✅ Fixed: ISIC_0000122.png
✅ Fixed: ISIC_0036333.png
✅ Fixed: ISIC_0000140.png
✅ Fixed: ISIC_0000032.png
✅ Fixed: ISIC_0019794.png
✅ Fixed: ISIC_0000036.png
✅ Fixed: ISIC_0000025.png
✅ Fixed: ISIC_0000113.png
✅ Fixed: ISIC_0000042.png
✅ Fixed: ISIC_0000086.png
✅ Fixed: ISIC_0019334.png
✅ Fixed: ISIC_0021449.png
✅ Fixed: ISIC_0017341.png
✅ Fixed: ISIC_0019723.png
✅ Fixed: ISIC_0036174.png
✅ Fixed: ISIC_0022147.png
✅ Fixed: ISIC_0000131.png
✅ Fixed: ISIC_0000141.png
✅ Fixed: ISIC_0024135.png
✅ Fixed: ISIC_0022039.png
✅ Fixed: ISIC_0000107.png
✅ Fixed: ISIC_0000088.png
✅ Fixed: ISIC_0000102.png
✅ Fixed: ISIC_0000026.png
✅ Fixed: ISIC_0015480.png
✅ Fixed: ISIC_0000097.png
✅ Fixed: ISIC_0000019.png
✅ Fixed: ISIC_0000112.png
✅ Fixed: ISIC_0000059.png
✅ Fixed: ISIC_0000004.png
✅ Fixed: ISIC_0000098.png
✅ Fixed: ISIC_0018680.png
✅ Fixed: ISIC_0017399.png
✅ Fixed: ISIC_0012623.png
✅ Fixed: ISIC_0000117.png
✅ Fixed: ISIC_0000119.png
✅ Fixed: ISIC_0000099.png
✅ Fixed: ISIC_0000038.png
✅ Fixed: ISI

In [1]:
from PIL import Image
import numpy as np
import os

def create_black_labels(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for fname in os.listdir(input_folder):
        if fname.endswith('.png'):
            input_path = os.path.join(input_folder, fname)
            output_path = os.path.join(output_folder, fname)
            try:
                # Load to get the size, then create black image
                original = Image.open(input_path)
                black = Image.fromarray(np.zeros(original.size[::-1], dtype=np.uint8))  # (H, W)
                black.save(output_path)
                print(f"✅ Saved black label: {fname}")
            except Exception as e:
                print(f"❌ Failed: {fname} ({e})")

# Example usage
create_black_labels(
    input_folder='/home/yi-yang/NNunet_isic/nnUNet_raw_data_base/nnUNet_raw/Dataset338_ISIC3/labelsTr2',
    output_folder='/home/yi-yang/NNunet_isic/nnUNet_raw_data_base/nnUNet_raw/Dataset338_ISIC3/labelsTr'
)


✅ Saved black label: ISIC_0010372.png
✅ Saved black label: ISIC_0013208.png
✅ Saved black label: ISIC_0014248.png
✅ Saved black label: ISIC_0013371.png
✅ Saved black label: ISIC_0015965.png
✅ Saved black label: ISIC_0015963.png
✅ Saved black label: ISIC_0013797.png
✅ Saved black label: ISIC_0000226.png
✅ Saved black label: ISIC_0015009.png
✅ Saved black label: ISIC_0014642.png
✅ Saved black label: ISIC_0012329.png
✅ Saved black label: ISIC_0011373.png
✅ Saved black label: ISIC_0013953.png
✅ Saved black label: ISIC_0014027.png
✅ Saved black label: ISIC_0013192.png
✅ Saved black label: ISIC_0000539.png
✅ Saved black label: ISIC_0010093.png
✅ Saved black label: ISIC_0010496.png
✅ Saved black label: ISIC_0011101.png
✅ Saved black label: ISIC_0014834.png
✅ Saved black label: ISIC_0015563.png
✅ Saved black label: ISIC_0010173.png
✅ Saved black label: ISIC_0000279.png
✅ Saved black label: ISIC_0010034.png
✅ Saved black label: ISIC_0013258.png
✅ Saved black label: ISIC_0014599.png
✅ Saved blac

In [8]:
from PIL import Image
import numpy as np
import os

def check_png_binary(path):
    try:
        img = Image.open(path)
        arr = np.array(img)
        unique_vals = np.unique(arr)
        print(f"{os.path.basename(path)} → unique values: {unique_vals}")
        return set(unique_vals).issubset({0, 1})
    except Exception as e:
        print(f"Error reading {path}: {e}")
        return False

# Example usage
is_binary = check_png_binary('/home/yi-yang/NNunet_isic/nnUNet_raw_data_base/nnUNet_raw/Dataset330_ISIC1/labelsTr/ISIC_0000000.png')
print("✅ Binary mask (0/1)" if is_binary else "❌ Not binary")


ISIC_0000000.png → unique values: [0 1]
✅ Binary mask (0/1)
